This is an interview assignment I got from someone who interviewed at BuildZoom. The data and objective seemed nice, so I gave it a try.

Objective:
- Build a classifier that predicts whether a building permit's 'type' is 'ELECTRICAL' or not. Note that there are many different types of permits but we are only interested in 'ELECTRICAL'.

# Notebook setup

In [1]:
import numpy as np
import pandas as pd

In [2]:
 # Disable trunkating the middle of long dataframes. I want to see all the columns.
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Load data

In [4]:
train_data = pd.read_csv('./data/train_data.csv', sep='\t')
test_data = pd.read_csv('./data/xtest_data.csv', sep='\t')

In [5]:
train_data['is_electrical'] = train_data.type == 'ELECTRICAL'

In [6]:
train_data.head()

,licensetype,businessname,legaldescription,description,type,subtype,job_value,is_electrical
0,NaN,ADT LLC,NaN,INSTALL LOW VOLTAGE SECURITY SYSTEM,ELECTRICAL,COMMERCIAL,NaN,True
1,NaN,NaN,NaN,INSTALL (1) NON-ILLUMINATED WALL SIGN - PLAZA GARIBALDI,SIGN/BILLBOARD,COMMERCIAL,NaN,False
2,SPECIALTY CONTRACTOR LICENSE,KLN MEDIA LLC,NaN,INSTALL (1) NON-ILLUM ON-PREMISES WALL SIGN FOR &quot;PLAZA GARIBALDI&quot;,SIGN/BILLBOARD,COMMERCIAL,NaN,False
3,"ELECTRICAL CONTRACTOR LICENSE, ELECTRICAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC., GEN...",OLSON ENERGY SERVICE,NaN,REPLACE OIL FURNACE W/ NEW GAS FURNACE,MECHANICAL /,SINGLE FAMILY / DUPLEX,NaN,False
4,NaN,NaN,NaN,WIRE NEW SINGLE FAMILY RESIDENCE W/ 200 AMP SERVICE,ELECTRICAL,SINGLE FAMILY / DUPLEX,NaN,True


# EDA

In [7]:
import locale

## Null values

In [8]:
train_data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100156 entries, 0 to 100155
Data columns (total 8 columns):
licensetype         36624 non-null object
businessname        72575 non-null object
legaldescription    24393 non-null object
description         98842 non-null object
type                87170 non-null object
subtype             72604 non-null object
job_value           15696 non-null object
is_electrical       100156 non-null bool
dtypes: bool(1), object(7)
memory usage: 5.4+ MB


- The dataset contains quite a high number of null values.
- There are even observations where each predictor is null. This offers no information so we can drop them.
- job_value is being treated as an object, most likely because they have $ in them
- Only one field is numerical, the others are all text, so we will most likely have to search for keywords in them.
- We are tasked with predicting the type field, but there are about 13,000 observations that are missing this value. This is unlabelled data, so those can be dropped.

In [9]:
print("Before dropping NaN only: ", len(train_data))
nan_only = train_data[train_data.isnull().all(axis=1)]
train_data.drop(nan_only.index, inplace=True)
train_data.reset_index(drop=True, inplace=True)
print("After dropping NaN only: ", len(train_data))

Before dropping NaN only:  100156
After dropping NaN only:  100156


In [17]:
test_data[test_data.isnull().all(axis=1)]

,licensetype,businessname,legaldescription,description,subtype,job_value,electrical_only
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are some objects which miss the "type" field, so it's unlabelled data. We can drop those.

In [10]:
unlabelled = train_data[train_data.type.isnull()]
train_data.drop(unlabelled.index, inplace=True)
train_data.reset_index(drop=True, inplace=True)
print("After dropping unlabelled data: ", len(train_data))

After dropping unlabelled data:  87170


In [11]:
train_data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87170 entries, 0 to 87169
Data columns (total 8 columns):
licensetype         36624 non-null object
businessname        72573 non-null object
legaldescription    24393 non-null object
description         86924 non-null object
type                87170 non-null object
subtype             72604 non-null object
job_value           15696 non-null object
is_electrical       87170 non-null bool
dtypes: bool(1), object(7)
memory usage: 4.7+ MB


Description and business name have almost no null values, it will be important to use this feature.

## Data types

job_value should be numerical, but it's an object. Let's fix that.

In [12]:
job_values = train_data[train_data.job_value.notnull()].job_value
(job_values.str.startswith('$') & job_values.str.endswith('.00')).unique()

array([ True])

In [13]:
train_data.job_value = train_data.job_value.str.strip('$')
train_data.job_value = train_data.job_value.apply(lambda x: x[:-3] if pd.notnull(x) else x)

In [14]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
converted = train_data[train_data.job_value.notnull()].job_value.apply(locale.atoi)
train_data.loc[train_data.job_value.notnull(), 'job_value'] = converted
del converted

All job_value start with a $ and end with .00 so we can safely strip those. 

# Feature Engineering

Ideas:
- Check which businesses do only electronic works
- Explore similarities between business names that do electronic works, which words occur often
- Explore description for similar words

## Only electric contracts

In [15]:
only_electrical = train_data.groupby('businessname').is_electrical.agg(np.all)
only_electrical.name = 'electrical_only'
train_data = train_data.join(only_electrical, on='businessname')
test_data = test_data.join(only_electrical, on='businessname')

In [16]:
train_data.head()

,licensetype,businessname,legaldescription,description,type,subtype,job_value,is_electrical,electrical_only
0,NaN,ADT LLC,NaN,INSTALL LOW VOLTAGE SECURITY SYSTEM,ELECTRICAL,COMMERCIAL,NaN,True,True
1,NaN,NaN,NaN,INSTALL (1) NON-ILLUMINATED WALL SIGN - PLAZA GARIBALDI,SIGN/BILLBOARD,COMMERCIAL,NaN,False,NaN
2,SPECIALTY CONTRACTOR LICENSE,KLN MEDIA LLC,NaN,INSTALL (1) NON-ILLUM ON-PREMISES WALL SIGN FOR &quot;PLAZA GARIBALDI&quot;,SIGN/BILLBOARD,COMMERCIAL,NaN,False,False
3,"ELECTRICAL CONTRACTOR LICENSE, ELECTRICAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC., GEN...",OLSON ENERGY SERVICE,NaN,REPLACE OIL FURNACE W/ NEW GAS FURNACE,MECHANICAL /,SINGLE FAMILY / DUPLEX,NaN,False,False
4,NaN,NaN,NaN,WIRE NEW SINGLE FAMILY RESIDENCE W/ 200 AMP SERVICE,ELECTRICAL,SINGLE FAMILY / DUPLEX,NaN,True,NaN
